In [ ]:
! pip install -U sagemaker

## Create an endpoint from hugginface_model

https://aws.amazon.com/blogs/machine-learning/configuring-autoscaling-inference-endpoints-in-amazon-sagemaker/

https://medium.com/p/b1b6e6731c59


In [ ]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'openpecha/wav2vec2_run9',
	'HF_TASK':'automatic-speech-recognition'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.37.0',
	pytorch_version='2.1.0',
	py_version='py310',
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge' # ec2 instance type
)

Check if the endpoint works

In [ ]:

from sagemaker.serializers import DataSerializer
	
predictor.serializer = DataSerializer(content_type='audio/x-audio')

# Make sure the input file exists
with open("/home/ec2-user/SageMaker/stt-split-audio/STT_NS/segments_ns/STT_NS_M0322_0400_2713489_to_2721015.wav", "rb") as f:
	data = f.read()
predictor.predict(data)

In [ ]:
use invoke_endpoint to run inference

In [ ]:
import os
import io
import boto3
import json
import csv

# grab environment variables
ENDPOINT_NAME = 'huggingface-pytorch-inference-2024-02-21-10-00-23-117'
runtime= boto3.client('runtime.sagemaker')

with open("/home/ec2-user/SageMaker/stt-split-audio/STT_NS/segments_ns/STT_NS_M0322_0400_2713489_to_2721015.wav", "rb") as f:
	data = f.read()

response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                      ContentType='audio/x-audio',
                                      Body=data)
    
result = json.loads(response['Body'].read().decode())
inference = {"inference": result['text']}
response_dict = {
          "statusCode": 200,
          "body": inference
                }
print(response_dict)

Check the endpoint description

In [ ]:
import pprint
import boto3
from sagemaker import get_execution_role
import sagemaker
import json

pp = pprint.PrettyPrinter(indent=4, depth=4)
role = get_execution_role()
sagemaker_client = boto3.Session().client(service_name='sagemaker')
endpoint_name = 'huggingface-pytorch-inference-2024-02-21-10-00-23-117'
response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
pp.pprint(response)

# make the endpoint autoscaling 

In [ ]:
client = boto3.client('application-autoscaling') # Common class representing Application Auto Scaling for SageMaker amongst other services

endpoint_name = 'huggingface-pytorch-inference-2024-02-21-10-00-23-117'
resource_id='endpoint/' + endpoint_name + '/variant/' + 'AllTraffic' # This is the format in which application autoscaling references the endpoint

response = client.register_scalable_target(
    ServiceNamespace='sagemaker', #
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    MinCapacity=1,
    MaxCapacity=5
)


#Example 1 - SageMakerVariantInvocationsPerInstance Metric
response = client.put_scaling_policy(
    PolicyName='Invocations-ScalingPolicy',
    ServiceNamespace='sagemaker', # The namespace of the AWS service that provides the resource. 
    ResourceId=resource_id, # Endpoint name 
    ScalableDimension='sagemaker:variant:DesiredInstanceCount', # SageMaker supports only Instance Count
    PolicyType='TargetTrackingScaling', # 'StepScaling'|'TargetTrackingScaling'
    TargetTrackingScalingPolicyConfiguration={
        'TargetValue': 10.0, # The target value for the metric. - here the metric is - SageMakerVariantInvocationsPerInstance
        'PredefinedMetricSpecification': {
            'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance', # is the average number of times per minute that each instance for a variant is invoked. 
        },
        'ScaleInCooldown': 600, # The cooldown period helps you prevent your Auto Scaling group from launching or terminating 
                                # additional instances before the effects of previous activities are visible. 
                                # You can configure the length of time based on your instance startup time or other application needs.
                                # ScaleInCooldown - The amount of time, in seconds, after a scale in activity completes before another scale in activity can start. 
        'ScaleOutCooldown': 300 # ScaleOutCooldown - The amount of time, in seconds, after a scale out activity completes before another scale out activity can start.
        
        # 'DisableScaleIn': True|False - ndicates whether scale in by the target tracking policy is disabled. 
                            # If the value is true , scale in is disabled and the target tracking policy won't remove capacity from the scalable resource.
    }
)

## check the scaling policy

In [ ]:
response = client.describe_scaling_policies(
    ServiceNamespace='sagemaker'
)

for i in response['ScalingPolicies']:
    print('')
    pp.pprint(i['PolicyName'])
    print('')
    if('TargetTrackingScalingPolicyConfiguration' in i):
        pp.pprint(i['TargetTrackingScalingPolicyConfiguration']) 
    else:
        pp.pprint(i['StepScalingPolicyConfiguration'])
    print('')

## see how many instances is running

In [ ]:
sagemaker_client = boto3.Session().client(service_name='sagemaker')
response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
status = response['EndpointStatus']
instance_count = response['ProductionVariants'][0]['CurrentInstanceCount']

print("Status: " + status)
print(f"Current Instance count: {instance_count}")

## increase or decrease the number of instances manually

In [ ]:
response = sagemaker_client.update_endpoint_weights_and_capacities(EndpointName=endpoint_name,
                            DesiredWeightsAndCapacities=[
                                {
                                    'VariantName': 'AllTraffic',
                                    'DesiredInstanceCount': 2
                                }
                            ])

### To use the inference endpoint we can create an API Gateway and Lambda function. The Lambda function can forward the call to the sagemaker endpoint.

https://www.linkedin.com/pulse/upload-files-aws-s3-api-gateway-lambda-adeolu-biobaku/

https://github.com/Spidy20/Sagemaker-Tutorials/blob/master/Tutorial%20-%202%20Create%20Rest%20API%20for%20Sagemaker%20Endpoint/Tutorial%20-%202%20Create%20Rest%20API.ipynb



For Authorization we can use AWS_IAM and give that user `stt-envoker` permission to call the API Gateway with `AmazonAPIGatewayInvokeFullAccess` policy. 

https://youtu.be/KXyATZctkmQ


### Lambda code to envoke the Sagemaker Endpoint

In Configuration > Permissions > Role name 
Go the the Role and attach the inline policy 

```json
{
	"Version": "2012-10-17",
	"Statement": [
		{
			"Sid": "VisualEditor0",
			"Effect": "Allow",
			"Action": "sagemaker:InvokeEndpoint",
			"Resource": "*"
		}
	]
}
```

To allow the Lambda function to  Invoke Sagemaker Endpoint

In [ ]:
import os
import io
import boto3
import json
import csv
import base64

# grab environment variables
ENDPOINT_NAME = os.environ['ENDPOINT_NAME']

runtime= boto3.client('runtime.sagemaker')


def lambda_handler(event, context):
    
    print('event: ' ,event)
    
    try:
        file_content = event["content"]
    except KeyError:
        return {
            "statusCode": 400,
            "error": "Audio file not found"
        }
    
    payload_data = base64.b64decode(file_content)
    
    # try:
    response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                      ContentType='audio/x-audio',
                                      Body=payload_data)
    # except Exception as e:
    #     return {
    #         "statusCode": 500,
    #         "error": "Sagemaker endpoint is down"
            
    #     }
    result = json.loads(response['Body'].read().decode())
    body = {
        "inference": result['text'],
        "model": "openpecha/wav2vec2_run9",
        "model_version": "9"
    }
    response_dict = {
          "statusCode": 200,
          "body": body,
                }
    return response_dict

### In the API Gateway

`integration request` > `Mapping templates` 

Content type: audio/x-audio

Template body
```json
{

    "content": "$input.body"

}
```

In the `API settings` > `Binary media types`
Add audio/x-audio 

# Get the audio from S3 and run inference

supply input like in the reqest body
```json
{
    "bucket": "monlam.ai.stt",
    "key": "test-for-sagemaker-endpoint/STT_NS_0060.wav"
}
```

Use proxy passthrough in API Gateway

In [ ]:
import json
import boto3
import botocore

def lambda_handler(event, context):
    # TODO implement
    
    print(event)
    
    s3 = boto3.client('s3')
    BUCKET_NAME = event['bucket']
    KEY = event['key']
    
    s3_response = s3.get_object(Bucket=BUCKET_NAME, Key=KEY)
    data = s3_response['Body'].read()
        # print(type(data))
        
    endpoint_name = 'huggingface-pytorch-inference-2024-02-27-07-16-05-752'
    runtime = boto3.client('runtime.sagemaker')
        
    sm_response = runtime.invoke_endpoint(EndpointName=endpoint_name,
                                      ContentType='audio/x-audio',
                                      Body=data)
    inf = json.loads(sm_response['Body'].read().decode())
    
    print(type(inf))
        
    return {
        'statusCode': 200,
        'body': inf
        }